In this notebook you'll create your own bootstrap function following the bootstrap algorithm (check the lecture notes!)

In [267]:
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np

In [268]:
# Load the data
df = pd.read_csv('/content/customers.csv')
data = df.values.T[1]

In [269]:
# Checking the notes from the lecture, create here your own bootstrap function:
# 1. Sample from the input array x to create an array of samples of shape (n_bootstraps, sample_size)
# Hint: Check the function random.choice() on Numpy
# 2. Calculate and save the mean of the array (this is "data_mean" that is returned by the function)
# 3. Calculate the mean from each bootstrap (i.e., row) and store it.
# (This should be an array of n_bootstraps values)
# 4. Calculate the lower and upper bounds for a 95% CI (hint: check the percentile function on Numpy)
# 5. Return data_mean, and the lower and upper bounds of your interval
def bootstrap_mean(x, sample_size, n_bootstraps):
  means = []
  for i in range(n_bootstraps):
    temp_list = []
    for n in range(sample_size):
      temp_list = np.append(temp_list, np.random.choice(x))
    means = np.append(means, temp_list.mean())
  data_mean = means.mean()
  lower = np.percentile(means, 2.5, interpolation='lower')
  upper = np.percentile(means, 97.5, interpolation='higher')

  return data_mean, lower, upper

In [ ]:
# Call your bootstrap function and plot the results

boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean(data, data.shape[0], i)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Bootstrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 100000)



Now, modify the bootstrap function you created above so that you can pass your desired confidence interval as a parameter.



In [ ]:
def bootstrap_mean_ci(sample, sample_size, n_bootstraps, ci):
  means = []
  for i in range(n_bootstraps):
    temp_list = []
    for n in range(sample_size):
      temp_list = np.append(temp_list, np.random.choice(sample))
    means = np.append(means, temp_list.mean())
  data_mean = means.mean()
  lower = np.percentile(means, (100 - ci) / 2, interpolation='lower')
  upper = np.percentile(means, 100 - ((100 - ci) / 2), interpolation='higher')
  
  return data_mean, lower, upper

In [ ]:
def bootstrap_sd_ci(sample, sample_size, n_bootstraps, ci):
  stds = []
  for i in range(n_bootstraps):
    temp_list = []
    for n in range(sample_size):
      temp_list = np.append(temp_list, np.random.choice(sample))
    stds = np.append(stds, np.std(temp_list))
  data_std = np.std(stds)
  lower = np.percentile(stds, (100 - ci) / 2, interpolation='lower')
  upper = np.percentile(stds, 100 - ((100 - ci) / 2), interpolation='higher')
  return data_std, lower, upper

In [ ]:
boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean_ci(data, data.shape[0], i, 80)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 100000)

#sns_plot.savefig("bootstrap_confidence_80.pdf", bbox_inches='tight')


# Vehicles dataset

Now let's work on a different dataset, which is stored in the vehicles.csv file.


In [ ]:
# Load and visualise the vehicles dataset
# To load the dataset: https://neptune.ai/blog/google-colab-dealing-with-files (check section "Load individual files directly from GitHub")


# Note that the current and new fleets are in different columns and have different lengths, so bear this in mind when you're plotting.
# You can create separate scatterplots for the two fleets, as you would with the histograms, 
# or plot them both in one plot (but not one against the other).
# <---INSERT YOUR CODE HERE--->
# Note: you can add more cells as needed to organise your code and your plots


In [ ]:
dfv = pd.read_csv('/content/vehicles.csv')
print("Shape:", dfv.shape)
print("NAN: \n", dfv.isna().sum())
dfv.head()

In [ ]:
sns.scatterplot(data=dfv, x=range(dfv.shape[0]), y="Current fleet")

In [ ]:
sns.scatterplot(data=dfv, x=range(dfv.shape[0]), y="New Fleet")

## Compare the two fleets

The business analysts come up a comparison algorithm that requires the upper and lower bounds for the mean in order to say which fleet is better.
1. Calculate the mean of both samples.
2. Using the bootstrap function that you created:
    - Construct the 95% CI of the mean of the current fleet.
    - Construct the 95% CI of the mean of the new fleet.
    - Are they comparable? (i.e., is one better than the other?) -- you can do this with a permutation test (check the lecture notes!)

In [ ]:
cfleet = dfv["Current fleet"]
nfleet = dfv["New Fleet"].dropna()

cfleet = np.array(cfleet)
nfleet = np.array(nfleet)

x_old = cfleet.mean()
x_new = nfleet.mean()
print(x_old)
print(x_new)
t_obs = x_new - x_old
print(t_obs)

In [ ]:
cf_boot = bootstrap_mean_ci(cfleet, cfleet.shape[0], 50000, 95)
print(cf_boot)

nf_boot = bootstrap_mean_ci(nfleet, nfleet.shape[0], 50000, 95)
print(nf_boot)

In [ ]:
# Create your own function for a permutation test here (you will need it for the lab quiz!):
def permut_test(sample1, sample2, n_permutations):
    """
    sample1: 1D array
    sample2: 1D array (note that the size of the two arrays can be different)
    n_permutations: number of permutations to calculate the p-value
    """
    concat = np.concatenate([sample1, sample2])
    counter = 0
    for i in range(n_permutations):
      perm = np.random.permutation(concat)
      pold = perm[:len(sample1)]
      pnew = perm[len(sample1):]
      x_perm_old = pold.mean()
      x_perm_new = pnew.mean()
      t_perm = x_perm_new - x_perm_old
      if (t_perm > t_obs):
        counter +=1
    pvalue = counter / n_permutations
    return pvalue

In [ ]:
permut_test(cfleet, nfleet, 30000)

Quiz Solutions

Q2

In [ ]:
cboot = bootstrap_mean_ci(cfleet, len(cfleet), 10000, 92)
print(cboot)

Q3

In [ ]:
cboot = bootstrap_mean_ci(nfleet, len(nfleet), 10000, 95)
print(cboot)

Q4

In [ ]:
cboot = bootstrap_sd_ci(nfleet, len(nfleet), 10000, 95)
print(cboot)

Q5

In [266]:
cboot = bootstrap_sd_ci(data, data.shape[0], 10000, 90)
print(cboot)

(1.1967258447797429, 3.0380243311623913, 7.109995831117439)


Q7

In [ ]:
permut_test(cfleet, nfleet, 30000)

In [ ]:
# The variables below represent the percentages of democratic votes in Pennsylvania and Ohio (one value for each state).
dem_share_PA = [60.08, 40.64, 36.07, 41.21, 31.04, 43.78, 44.08, 46.85, 44.71, 46.15, 63.10, 52.20, 43.18, 40.24, 39.92, 47.87, 37.77, 40.11, 49.85, 48.61, 38.62, 54.25, 34.84, 47.75, 43.82, 55.97, 58.23, 42.97, 42.38, 36.11, 37.53, 42.65, 50.96, 47.43, 56.24, 45.60, 46.39, 35.22, 48.56, 32.97, 57.88, 36.05, 37.72, 50.36, 32.12, 41.55, 54.66, 57.81, 54.58, 32.88, 54.37, 40.45, 47.61, 60.49, 43.11, 27.32, 44.03, 33.56, 37.26, 54.64, 43.12, 25.34, 49.79, 83.56, 40.09, 60.81, 49.81]
dem_share_OH = [56.94, 50.46, 65.99, 45.88, 42.23, 45.26, 57.01, 53.61, 59.10, 61.48, 43.43, 44.69, 54.59, 48.36, 45.89, 48.62, 43.92, 38.23, 28.79, 63.57, 38.07, 40.18, 43.05, 41.56, 42.49, 36.06, 52.76, 46.07, 39.43, 39.26, 47.47, 27.92, 38.01, 45.45, 29.07, 28.94, 51.28, 50.10, 39.84, 36.43, 35.71, 31.47, 47.01, 40.10, 48.76, 31.56, 39.86, 45.31, 35.47, 51.38, 46.33, 48.73, 41.77, 41.32, 48.46, 53.14, 34.01, 54.74, 40.67, 38.96, 46.29, 38.25, 6.80, 31.75, 46.33, 44.90, 33.57, 38.10, 39.67, 40.47, 49.44, 37.62, 36.71, 46.73, 42.20, 53.16, 52.40, 58.36, 68.02, 38.53, 34.58, 69.64, 60.50, 53.53, 36.54, 49.58, 41.97, 38.11]

Q8.1

In [ ]:
print("dem_share_PA ", len(dem_share_PA))
print("dem_share_OH ", len(dem_share_OH))

Q8.2

In [ ]:
cboot = bootstrap_mean_ci(dem_share_OH, len(dem_share_OH), 200000, 95)
print(cboot)

Q8.3

In [ ]:
cboot = bootstrap_mean_ci(dem_share_PA, len(dem_share_PA), 200000, 95)
print(cboot)

Q8.4

In [ ]:
permut_test(dem_share_PA, dem_share_OH, 10000)